# BUILD LLM Bootcamp Day 2

In [1]:
# !pip install --no-deps --force-reinstall snowflake_ml_python-1.0.12-py3-none-any.whl

In [2]:
# !pip install peft transformers tokenizers vllm bitsandbytes datasets absl-py

In [3]:
# TODO: Set your Hugging Face token
# !huggingface-cli login --token YOUR_HG_TOKEN_GOES_HERE

### Import Libraries

In [1]:
from snowflake.snowpark.session import Session
from snowflake.ml.model.models import llm
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
import snowflake.ml.playground.finetune as ft_util
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F

import sys
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Establish Secure Connection

*NOTE: Update connection.json file with your credentials*

In [2]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : DASH
Role                        : "DASH_SPCS"
Database                    : "DASH_DB"
Schema                      : "DASH_SCHEMA"
Warehouse                   : "DASH_L"
Snowflake version           : 7.40.0
Snowpark for Python version : 1.9.0


### Fine-tune Llama 2 in Snowpark Container Services

#### Load Training Dataset

In [3]:
df = pd.read_json("transcripts.json", lines=True)
print(f"Total records: {df.shape[0]}")
# In the interest of time we will train, eval and test on 100 records
train_df = df.head(100)
print(f"Train        : {train_df.shape[0]}")
eval_df = df[200:300]
print(f"Eval         : {eval_df.shape[0]}")
test_df = df.tail(100)
print(f"Test         : {test_df.shape[0]}")

Total records: 990
Train        : 100
Eval         : 100
Test         : 100


#### Fine-tune Llama 2 Model

Fine Tuning is one form of model training. We start training from a pre-trained model and adjust a set of model parameters to better solve a concrete task based on task specific data.
We will be doing lora finetune today which is a parameter efficient way of fine tuning. 

Instead of adjusting all the ~7B parameters, LoRA (Low-Rank Adaptation) allows us to adjust only single digit percent of model weights--which can save compute and memory dramatically. For this lab, we will finetune a 7b llama2 model using LoRA on a single A10 GPU. This also demostrates how good the inference can be on finetuned models even with limited compute.

We're also passing `train_dataset` and `eval_dataset` that are used to generate loss calculation along the way. And for `train`, `output_weights_dir` is the directory where the finetune weights are stored after training completion.

For configuration, for quick illustration, we will just set `max_steps`=1 with 4bit quantization. Quantization is required to train on single A10.  To achieve good performance for the task, you will need at least 1 `num_epochs`, feel free to explore that on your own.

In [4]:
!mkdir merged_weights_dir
!mkdir output_weights_dir

mkdir: cannot create directory ‘merged_weights_dir’: File exists
mkdir: cannot create directory ‘output_weights_dir’: File exists


In [5]:
%%time
# Takes about ~5mins for 100 records

merged_weights_dir = 'merged_weights_dir'
output_weights_dir = 'output_weights_dir'

runner = ft_util.FinetuneModelRunner(
    base_model_id="meta-llama/Llama-2-7b-chat-hf",
    task_type=ft_util.FinetuneTaskType.INSTRUCT,
    train_dataset=train_df,
    eval_dataset=eval_df,
)

mcf = ft_util.ModelConfig(
    max_steps=1,
    quantization=ft_util.QuantizationConfig(
        k_bits=4,
    ),
    merged_weights_dir_path=merged_weights_dir,
)

_ = runner.train(output_weights_dir, model_cfg=mcf)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


Map (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Detected kernel version 5.4.181, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO 11-13 23:35:32 llm_engine.py:72] Initializing an LLM engine with config: model='merged_weights_dir', tokenizer='merged_weights_dir', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 11-13 23:36:23 llm_engine.py:207] # GPU blocks: 864, # CPU blocks: 512
CPU times: user 49.6 s, sys: 53.2 s, total: 1min 42s
Wall time: 2min 41s


#### Evaluate Results

After the training job completes, we call `get_eval_results`, this will run evaluation and get model predictions using the entire `eval_dataset`. You can see the output by model in the `predicted` column of the dataframe. This allows you to understand the performance of the current model better.

*Examine the "output" and "predicted" columns*

In [7]:
eval_df = runner.get_eval_results()
eval_df.head()

Processed prompts: 100%|██████████| 100/100 [00:23<00:00,  4.28it/s]
/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/playground/finetune.py:573: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df["predicted"] = ts


,instruction,input,output,language,predicted
200,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hello! It's Frosty here to make your holiday wishes come true. May I know your name?\ncaller: My name's Oliver, Frosty.\nfrosty: Hi Oliver! Where are you from?\ncaller: From Manchester in the UK.\nfrosty: Great! And what toy are you wishing for?\ncaller: Um, I think I want a snap circuit set or a magic microscope but I'm not sure which.\nfrosty: Snap Circuits: Junior and LeapFrog Magic Adventures Microscope. Those are both educational and fun. Is that why you're interested?\ncaller: Yes, I like learning new things while playing. I think I'll go with the snap circuit set.\nfrosty: I can see why you'd choose it, it's a popular choice. And before we finish, can you tell me how you plan to celebrate the holiday season?\ncaller: I'm going to have a sleepover with my cousins! It's our tradition.\nfrosty: That sounds great! So, the Snap Circuits: Junior for your holiday wish, right Oliver?\ncaller: Yes, please, Frosty. Thank you.\nfrosty: Certainly, Oliver. Happy holidays!""","{""toy_list"": [""Snap Circuits: Junior""], ""location"": ""Manchester""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""Manchester"",\n""toy_list"": [\n""Snap Circuits: Junior"",\n""LeapFrog Magic Adventures Microscope""\n]\n}"
201,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hi friend! I'm here to help make your holiday joyous. What's your name?\ncaller: Hello Frosty, I'm Charlotte!\nfrosty: Hi Charlotte, where are you from?\ncaller: I'm from Toronto, Canada.\nfrosty: Awesome! What's on your holiday wish list, Charlotte?\ncaller: I really can't decide between the magna tiles set and the fort building kit. They both seem so fun.\nfrosty: Interesting choices! Magna-Tiles 32-piece set and Tiny Land kids fort building kit. Do you like these because you love building and designing?\ncaller: Yes, I love creating my own world! You know what? I'll go with the fort building kit. It seems more exciting!\nfrosty: Good choice, Charlotte! Before we finish, can you tell me what you love most about this time of the year?\ncaller: Yes, I love the holiday lights and spending time with my family.\nfrosty: Wonderful! It's the Tiny Land kids fort building kit then. Happy holidays, Charlotte!""","{""toy_list"": [""Tiny Land kids fort building kit""], ""location"": ""Toronto""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""Toronto, Canada"",\n""toy_list"": [\n""Magna-Tiles 32-piece set"",\n""Tiny Land kids fort building kit""\n]\n}"
202,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""Frosty: Hi there, welcome to our wonderful world of winter wishes! How may I help you today?\nCaller: Hi! My name is Emma, I'm ten.\nFrosty: Hello Emma, it's nice to meet you! Where are you calling from?\nCaller: I'm from Los Angeles, California.\nFrosty: It sounds like it's warm there while we're in a winter wonderland! Well, Emma, are you ready to tell Frosty what you would like as your holiday presents this year?\nCaller: Umm, I think I want the fox plushie and the baby bee doll.\nFrosty: Oh, those are amazing choices! Why are you interested in these toys, Emma?\nCaller: They are so soft and cute. I love cuddling with plushies!\nFrosty: Great choices! Plushies are indeed very cuddly. How do you and your family plan to celebrate the holiday season?\nCaller: We will be baking cookies and watching movies together.\nFrosty: That sounds wonderful! Alright Emma, I've got your wish for the 2023 Holiday Fox 12-Inch Plush and Orijin Bees Lovey Coiley Baby Bee. Thank you for your call!

#### Inference on Custom Transcript

Additionally, you may want to manually craft some cases to test the boundary of the model. To do that, you could use the `infer` method to run prediction against a single sample interactively.

In [8]:
runner.infer(
     {
    'instruction': """
        "Extract JSON response with 'location' and 'toy_list' as keys. 'location': 
        Location of the caller. Include city only.'toy_list': List of toy names from the caller."
    """,
    'input': """
        "frosty: Hello, hello! Who's ready to chat with Frosty today?\ncaller: Hi, I'm Dustin from Cape Town, South Africa.\nfrosty: 
        Hi Dustin, it's awesome to meet you! Do you already know what you would like for your holiday gift?\ncaller: 
        Yeah, I think I want the ninja turtles delivery van, or maybe the robot dog. 
        I can't decide...\nfrosty: Both sound like entertaining choices! 
        Why are you interested in these particular toys?\ncaller: The ninja turtles van is so cool! And the robot dog, it's like a pet you don't have to clean up after.\nfrosty: 
        That's true! Savage choices, Dustin! How do you usually celebrate the holiday season?\ncaller: 
        We usually have a big family barbeque and play games all day long.\nfrosty: Sounds fun! What is your favorite thing about this time of year?
        \ncaller: I like that everyone is happy and I get to spend a lot of time with my family.
        \nfrosty: That's wonderful! So Dustin, have you thought any more about your holiday wish?
        \ncaller: I think I'm going to get the robot dog. It sounds fun and I won't have to walk it.
        \nfrosty: That's a clever choice, Dustin! I hope you have a fantastic holiday season!"
"""
    }
)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.58s/it]


'Here is the JSON response with the requested keys:\n\n{\n"location": "Cape Town",\n"toy_list": ["ninja turtles delivery van", "robot dog"]\n}'

#### Inference on Test Dataset

Model performance is usually measured against the holdout test dataset. For that, you could use `infer_batch` to run batch inference against a dataframe.

*Examine the "output" and "predicted" columns*

In [9]:
test_res = runner.infer_batch(test_df)
test_res.head()

Processed prompts: 100%|██████████| 100/100 [00:32<00:00,  3.08it/s]
/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/playground/finetune.py:639: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["predicted"] = ts


,instruction,input,output,language,predicted
890,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hi! Welcome to the Holiday Wish Hotline, I'm Frosty! What's your name?\ncaller: My name's Bella!\nfrosty: Hello, Bella! It's great to have you here, where are you calling from?\ncaller: I'm calling from Boston, USA.\nfrosty: Wonderful, Bella! How old are you?\ncaller: I'm 7 years old.\nfrosty: That's awesome! Now, let's talk about your holiday wish list. Are there any toys from our list you're interested in?\ncaller: Um, I like the slime kit and playfoam letters.\nfrosty: What a fun combination! The National Geographic Mega Slime Kit & Putty Lab and Playfoam Naturals Shape & Learn Letters & Numbers are both great choices. Why are you interested in these toys?\ncaller: I love making squishy things and learning at the same time.\nfrosty: That's fantastic! What do you and your family plan to do to celebrate the holiday season, Bella?\ncaller: We'll bake cookies and play board games together. It's always so much fun!\nfrosty: That sounds like a wonderful time! So, you would like to add the slime kit and playfoam letters to your holiday wish list, right?\ncaller: Yes, please!\nfrosty: Your wish list will include the National Geographic Mega Slime Kit & Putty Lab and Playfoam Naturals Shape & Learn Letters & Numbers. Have a fantastic holiday season, Bella, and enjoy playing with your new toys!\ncaller: Thank you, Frosty! Bye!""","{""toy_list"": [""National Geographic Mega Slime Kit & Putty Lab"", ""Playfoam Naturals Shape & Learn Letters & Numbers""], ""location"": ""Boston""}",EN,"Here is the JSON response with ""location"" and ""toy_list"" as keys:\n\n{\n""location"": ""Boston, USA"",\n""toy_list"": [\n""National Geographic Mega Slime Kit & Putty Lab"",\n""Playfoam Naturals Shape & Learn Letters & Numbers""\n]\n}"
891,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty: Hey there!Welcome to the Holiday Wish Hotline! My name is Frosty! What's your name?\ncaller: Hi Frosty! My name is Danny.\nfrosty: Nice to meet you, Danny! Where are you calling from?\ncaller: I'm from Vancouver, Canada.\nfrosty: That's amazing! How old are you, Danny?\ncaller: I'm 10 years old.\nfrosty: Great! Let's talk about your holiday wish list. What toys do you want?\ncaller: Hmm, I'm not sure yet. I like building things and stuff.\nfrosty: If you're into building toys, you might enjoy the Tiny Land kids fort building kit. Or, if you're into technology, the Dog-E robot dog is popular this year. Why don't you tell me more about your hobbies and we can find the perfect toy for you?\ncaller: I like making stuff with my hands, like building forts with blankets and chairs. I also like playing video games with my friends.\nfrosty: That's fantastic, Danny! It sounds like the Tiny Land kids fort building kit would be perfect for you since you love building forts. How does that sound?\ncaller: Yeah, that sounds cool. I think I'll add the fort building kit to my wish list.\nfrosty: Great! Let's confirm your wish, Danny. You would like the Tiny Land kids fort building kit, right?\ncaller: Yes, please! That's perfect.\nfrosty: Fantastic! Enjoy your holiday season, Danny! Have fun building awesome forts!\ncaller: Thanks, Frosty! Happy holidays!""","{""toy_list"": [""Tiny Land kids fort building kit""], ""location"": ""Vancouver""}",EN,"Here is the JSON response that extracts the ""location"" and ""toy_list"" keys from the conversation:\n\n{\n""location"": ""Vancouver, Canada"",\n""toy_list"": [\n""Tiny Land kids fort building kit"",\n""Dog-E robot dog""\n]\n}"
892,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"""frosty

### Log and Deploy Fine-tuned Llama 2

In [10]:
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION = "V1.0"
MODEL_REGISTRY_DB = "DASH_DB"
MODEL_REGISTRY_SCHEMA = "DASH_SCHEMA"
COMPUTE_POOL = "DASH_BUILD_GPU3"
DEPLOYMENT_NAME = "TRAINED_LLAMA2"

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


In [11]:
registry.list_deployments(model_name=MODEL_NAME,model_version=MODEL_VERSION).to_pandas()

,MODEL_NAME,MODEL_VERSION,DEPLOYMENT_NAME,CREATION_TIME,TARGET_METHOD,TARGET_PLATFORM,SIGNATURE,OPTIONS,STAGE_PATH,ROLE


In [12]:
# registry.delete_deployment(model_name=MODEL_NAME,model_version=MODEL_VERSION,deployment_name=DEPLOYMENT_NAME)

In [13]:
registry.list_models().to_pandas()

,CREATION_CONTEXT,CREATION_ENVIRONMENT_SPEC,CREATION_ROLE,CREATION_TIME,ID,INPUT_SPEC,NAME,OUTPUT_SPEC,RUNTIME_ENVIRONMENT_SPEC,TYPE,URI,VERSION,ARTIFACT_IDS,DESCRIPTION,METRICS,TAGS,REGISTRATION_TIMESTAMP


In [14]:
# registry.delete_model(model_name=MODEL_NAME,model_version=MODEL_VERSION,delete_artifact=True)

*NOTE: Notice that the "model_id_or_path" is set to the output weights folder where the weights from fine-tuning the model are stored*

In [16]:
%%time
# NOTE: This will take ~15mins
options = llm.LLMOptions(
    token=connection_parameters["huggingface_token"],
    max_batch_size=100,
)

llama_model = llm.LLM(
    model_id_or_path=output_weights_dir,
    options=options
)

llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
    options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/_internal/env_utils.py:217: UserWarning: Package requirement absl-py<2,>=0.15 specified, while version 2.0.0 is installed. Local version will be ignored to conform to package requirement.
  warnings.warn(
/opt/conda/envs/rapids/lib/python3.8/site-packages/snowflake/ml/model/_packager/model_env/model_env.py:353: UserWarning: Found dependencies specified as pip requirements. This may prevent model deploying to Snowflake Warehouse.
  warnings.warn(


CPU times: user 2.44 s, sys: 112 ms, total: 2.55 s
Wall time: 15min 31s


{'name': 'DASH_DB.DASH_SCHEMA.TRAINED_LLAMA2',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'DASH_BUILD_GPU3', 'num_gpus': 1},
 'details': {'image_name': 'sfsenorthamerica-build-spcs.registry.snowflakecomputing.com/dash_db/dash_schema/snowml_repo/8a1453d3c1d39172488042f22e4a323c912464a3:latest',
  'service_spec': "spec:\n  container:\n  - env:\n      MODEL_ZIP_STAGE_PATH: /DASH_DB.DASH_SCHEMA.SNOWML_MODEL_6957FAF8827E11EEAF8046C69673BB50/model.zip\n      NUM_WORKERS: 1\n      SNOWML_USE_GPU: true\n      TARGET_METHOD: infer\n      _CONCURRENT_REQUESTS_MAX: 1\n    image: sfsenor

In [17]:
begin_prompt = \
"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
Extract JSON response with 'location' and 'toy_list' as keys.
'location': Location string of the caller.
'toy_list': List of toy names from the caller.
### Input:

"""
end_prompt = " [/INST]"

eval_df['input'] = begin_prompt + eval_df['input'] + end_prompt
eval_df = eval_df.reset_index(drop=True)
eval_df.head()

/tmp/ipykernel_1904/1994325739.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df['input'] = begin_prompt + eval_df['input'] + end_prompt


,instruction,input,output,language,predicted
0,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys.\n'location': Location string of the caller.\n'toy_list': List of toy names from the caller.\n### Input:\n\n""frosty: Hello! It's Frosty here to make your holiday wishes come true. May I know your name?\ncaller: My name's Oliver, Frosty.\nfrosty: Hi Oliver! Where are you from?\ncaller: From Manchester in the UK.\nfrosty: Great! And what toy are you wishing for?\ncaller: Um, I think I want a snap circuit set or a magic microscope but I'm not sure which.\nfrosty: Snap Circuits: Junior and LeapFrog Magic Adventures Microscope. Those are both educational and fun. Is that why you're interested?\ncaller: Yes, I like learning new things while playing. I think I'll go with the snap circuit set.\nfrosty: I can see why you'd choose it, it's a popular choice. And before we finish, can you tell me how you plan to celebrate the holiday season?\ncaller: I'm going to have a sleepover with my cousins! It's our tradition.\nfrosty: That sounds great! So, the Snap Circuits: Junior for your holiday wish, right Oliver?\ncaller: Yes, please, Frosty. Thank you.\nfrosty: Certainly, Oliver. Happy holidays!"" [/INST]","{""toy_list"": [""Snap Circuits: Junior""], ""location"": ""Manchester""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""Manchester"",\n""toy_list"": [\n""Snap Circuits: Junior"",\n""LeapFrog Magic Adventures Microscope""\n]\n}"
1,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'location' and 'toy_list' as keys.\n'location': Location string of the caller.\n'toy_list': List of toy names from the caller.\n### Input:\n\n""frosty: Hi friend! I'm here to help make your holiday joyous. What's your name?\ncaller: Hello Frosty, I'm Charlotte!\nfrosty: Hi Charlotte, where are you from?\ncaller: I'm from Toronto, Canada.\nfrosty: Awesome! What's on your holiday wish list, Charlotte?\ncaller: I really can't decide between the magna tiles set and the fort building kit. They both seem so fun.\nfrosty: Interesting choices! Magna-Tiles 32-piece set and Tiny Land kids fort building kit. Do you like these because you love building and designing?\ncaller: Yes, I love creating my own world! You know what? I'll go with the fort building kit. It seems more exciting!\nfrosty: Good choice, Charlotte! Before we finish, can you tell me what you love most about this time of the year?\ncaller: Yes, I love the holiday lights and spending time with my family.\nfrosty: Wonderful! It's the Tiny Land kids fort building kit then. Happy holidays, Charlotte!"" [/INST]","{""toy_list"": [""Tiny Land kids fort building kit""], ""location"": ""Toronto""}",EN,"Here is the JSON response with the requested keys:\n\n{\n""location"": ""Toronto, Canada"",\n""toy_list"": [\n""Magna-Tiles 32-piece set"",\n""Tiny Land kids fort building kit""\n]\n}"
2,Extract JSON response with 'location' and 'toy_list' as keys. 'location': Location of the caller. Include city only.'toy_list': List of toy names from the caller.,"\n[INST] <<SYS>>\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n<</SYS>>\n### Instruction:\nExtract JSON response with 'loca

In [18]:
llama_model_ref.predict(deployment_name=DEPLOYMENT_NAME,data=eval_df).head()

,generated_text
0,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Manchester, UK"",\n""toy_list"": [\n""Snap Circuits: Junior"",\n""LeapFrog Magic Adventures Microscope""\n]\n}"
1,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Toronto, Canada"",\n""toy_list"": [\n""Magna-Tiles 32-piece set"",\n""Tiny Land kids fort building kit""\n]\n}"
2,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Los Angeles, California"",\n""toy_list"": [\n""Fox plushie"",\n""Baby bee doll""\n]\n}"
3,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Manchester, England"",\n""toy_list"": [\n""Bluey Convertible"",\n""Figures"",\n""Dog-E""\n]\n}"
4,"Sure! Here is the JSON response with 'location' and 'toy_list' as keys:\n\n{\n""location"": ""Sydney, Australia"",\n""toy_list"": [\n""barbie science doll set"",\n""barbie dreamhouse""\n]\n}"


#### Run Inference in SQL

In [19]:
# Go to Snowsight

#### Clean Up Resources

Delete deployment and the model

In [20]:
llama_model_ref.delete_deployment(deployment_name=DEPLOYMENT_NAME)
llama_model_ref.delete_model(delete_artifact=True)